In [14]:
!pip install --upgrade openai

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
from openai import OpenAI
from tqdm import tqdm
import pandas as pd
import json

In [17]:
# Load the API key from the file
folder = "/content/drive/MyDrive/gpt"
with open(f'{folder}/gpt_key.txt', 'r') as f:
    openai_api_key = f.read().strip()
openai_client = OpenAI(api_key=openai_api_key)

In [18]:
def load_data(filepath):
    with open(filepath, 'r') as f:
        raw_data = json.load(f)
    records = []
    for entry in raw_data:
        sentence = " ".join(entry['tokens']).strip()
        ner_tag = " ".join(entry['ner_tags']).strip().lower()
        records.append({
            "sentence": sentence,
            "ner_tag": ner_tag
        })
    return pd.DataFrame(records)

# ARGUMENT DETECTION

In [ ]:
instructions = ("You are an argument analysis assistant. Your task is to identify and label argumentative spans in the input sentence according to the following types: claim and premise. A claim expresses a stance, opinion, or proposed policy, while a premise provides justification or support for a claim. "
        "Rewrite the entire sentence without changing or omitting any words. Surround each identified span with a tag indicating its type, using the format <claim>...</claim> or <premise>...</premise>."
        "Only apply tags to complete spans of meaning, not to partial words or fragments. Do not tag anything that does not clearly belong to one of the listed types. Output only the fully tagged sentence, do not include any additional text.")
examples = (
        """Examples:
        1) Sentence: Yes, I voted for it, supported it. Output: <premise>Yes, I voted for it</premise>, <claim>supported it.</claim>
        2) Sentence: Next question here for President Clinton. Yes, ma'am, here on the front row. Output: Next question here for President Clinton. Yes, ma'am, here on the front row.
        3) Sentence: Not some of the military. That was the decision of the Joint Chiefs of Staff, recommended to us and agreed to by the president. That is a fact. Output: <claim>Not some of the military.</claim> <premise>That was the decision of the Joint Chiefs of Staff, recommended to us and agreed to by the president. That is a fact.</premise>
        """
)

In [ ]:
examples

"Examples:\n        1) Sentence: Yes, I voted for it, supported it. Output: <premise>Yes, I voted for it</premise>, <claim>supported it.</claim>\n        2) Sentence: Next question here for President Clinton. Yes, ma'am, here on the front row. Output: Next question here for President Clinton. Yes, ma'am, here on the front row.\n        3) Sentence: Not some of the military. That was the decision of the Joint Chiefs of Staff, recommended to us and agreed to by the president. That is a fact. Output: <claim>Not some of the military.</claim> <premise>That was the decision of the Joint Chiefs of Staff, recommended to us and agreed to by the president. That is a fact.</premise>\n        "

In [ ]:
dataset = load_data(f"{folder}/argument_detection/test.json")
len(dataset)

2436

In [ ]:
dataset.head(2)

,sentence,ner_tag
0,Ohio had the best year it's ever had last year...,b-claim i-claim i-claim i-claim i-claim i-clai...
1,Yes?,o


## Zero-Shot

In [ ]:
print(instructions)

You are an argument analysis assistant. Your task is to identify and label argumentative spans in the input sentence according to the following types: claim and premise. A claim expresses a stance, opinion, or proposed policy, while a premise provides justification or support for a claim. Rewrite the entire sentence without changing or omitting any words. Surround each identified span with a tag indicating its type, using the format <claim>...</claim> or <premise>...</premise>.Only apply tags to complete spans of meaning, not to partial words or fragments. Do not tag anything that does not clearly belong to one of the listed types. Output only the fully tagged sentence, do not include any additional text.


In [ ]:
dataset.head(2)

,sentence,ner_tag
0,Ohio had the best year it's ever had last year...,b-claim i-claim i-claim i-claim i-claim i-clai...
1,Yes?,o


In [ ]:
results = []
engine = "gpt-4.1-nano"
max_token = 512
temperature = 0.0  # Lower temp for consistency

for idx, (sentence, tag) in tqdm(dataset.iterrows(), total=len(dataset)):
  response = openai_client.chat.completions.create(
      model=engine,
      messages=[{"role": "system", "content": instructions},
                {"role": "user", "content": f"Sentence: {sentence}"}],
      max_tokens=max_token,
      temperature=temperature,
  ).choices[0].message.content.strip()
  results.append({
      'sentence': sentence,
      'tags': tag,
      'gpt_response': response
  })

pd.DataFrame(results).to_csv(f"{folder}/argument_detection_zero_shot_results.csv", index=False)

100%|██████████| 2436/2436 [49:18<00:00,  1.21s/it]


In [ ]:
results[0]

{'sentence': "Ohio had the best year it's ever had last year. Michigan had the best year they've ever had.",
 'tags': 'b-claim i-claim i-claim i-claim i-claim i-claim i-claim i-claim i-claim i-claim o o o o o o o o',
 'gpt_response': "Ohio had the <claim>best year it's ever had last year</claim>. Michigan had the <claim>best year they've ever had</claim>."}

## Few-Shot

In [ ]:
instuctions_examples = f"{instructions}\n{examples.strip()}"
print(instuctions_examples)

In [ ]:
results = []
engine = "gpt-4.1-nano"
max_token = 512
temperature = 0.0  # Lower temp for consistency

for idx, (sentence, tag) in tqdm(dataset.iterrows(), total=len(dataset)):
  response = openai_client.chat.completions.create(
      model=engine,
      messages=[{"role": "system", "content": instuctions_examples},
                {"role": "user", "content": f"Sentence: {sentence}"}],
      max_tokens=max_token,
      temperature=temperature,
  ).choices[0].message.content.strip()
  results.append({
      'sentence': sentence,
      'tags': tag,
      'gpt_response': response
  })

pd.DataFrame(results).to_csv(f"{folder}/argument_detection_few_shot_results.csv", index=False)

# NER

In [ ]:
instructions = (
        "You are an information extraction assistant. Your task is to identify and label named entities in the input sentence according to the following types: politician, person, organization, politicalparty, event, election, country, location, and misc."
        "Rewrite the entire sentence without changing or omitting any words. Surround each identified entity with a tag indicating its type, using the format <type>...</type>. For example, tag a politician as <politician>...</politician>."
        "Only apply tags to complete spans of meaning, not to partial words or fragments. Do not tag anything that does not clearly belong to one of the listed types. Output only the fully tagged sentence, do not include any additional text."
)
examples = (
    """Examples:
        1) Sentence: Brazil's major cities began to resemble 1932-33 Berlin with its street battles between the Communist Party of Germany and the Nazi Party.
        Output: <country>Brazil</country>'s major cities began to resemble 1932-33 <location>Berlin</location> with its street battles between the <politicalparty>Communist Party of Germany</politicalparty> and the <politicalparty>Nazi Party</politicalparty>.

        2) Sentence: Amnesty International and Human Rights Watch as well as B'Tselem and Yesh Din criticized the military investigation.
        Output: <organization>Amnesty International</organization> and <organization>Human Rights Watch</organization> as well as <organization>B'Tselem</organization> and <organization>Yesh Din</organization> criticized the military investigation.

        3) Sentence: The People's Party, led by Aznar, won the most parliamentary seats at the 1996 Spanish general election, but he failed to obtain a majority in the Congress of Deputies, which forced the PP to seek the support of Basque (Basque Nationalist Party), Catalan (Convergence and Union) and Canarian (Canarian Coalition) regionalists.
        Output: <politicalparty>The People's Party</politicalparty>, led by <politician>Aznar</politician>, won the most parliamentary seats at the <election>1996 Spanish general election</election>, but he failed to obtain a majority in the <organization>Congress of Deputies</organization>, which forced the PP to seek the support of <misc>Basque</misc> (<politicalparty>Basque Nationalist Party</politicalparty>), <misc>Catalan</misc> (<politicalparty>Convergence and Union</politicalparty>) and <misc>Canarian</misc> (<politicalparty>Canarian Coalition</politicalparty>) regionalists.
        """
)

In [ ]:
print(instructions)

You are an information extraction assistant. Your task is to identify and label named entities in the input sentence according to the following types: politician, person, organization, politicalparty, event, election, country, location, and misc.Rewrite the entire sentence without changing or omitting any words. Surround each identified entity with a tag indicating its type, using the format <type>...</type>. For example, tag a politician as <politician>...</politician>.Only apply tags to complete spans of meaning, not to partial words or fragments. Do not tag anything that does not clearly belong to one of the listed types. Output only the fully tagged sentence, do not include any additional text.


In [ ]:
dataset = load_data(f"{folder}/ner/test.json")
len(dataset)

651

In [ ]:
dataset.head(2)

,sentence,ner_tag
0,"In the summer of 1914 , he took part in the me...",o o o o o o o o o o o o o o b-organisation i-o...
1,Cantwell was reelected in 2006 United States S...,b-politician o o o b-election i-election i-ele...


## Zero-Shot

In [ ]:
results = []
engine = "gpt-4.1-nano"
max_token = 512
temperature = 0.0  # Lower temp for consistency

for idx, (sentence, tag) in tqdm(dataset.iterrows(), total=len(dataset)):
  response = openai_client.chat.completions.create(
      model=engine,
      messages=[{"role": "system", "content": instructions},
                {"role": "user", "content": f"Sentence: {sentence}"}],
      max_tokens=max_token,
      temperature=temperature,
  ).choices[0].message.content.strip()
  results.append({
      'sentence': sentence,
      'tags': tag,
      'gpt_response': response
  })

pd.DataFrame(results).to_csv(f"{folder}/ner_zero_shot_results.csv", index=False)

100%|██████████| 651/651 [11:32<00:00,  1.06s/it]


In [ ]:
results

[{'sentence': "In the summer of 1914 , he took part in the meetings of the Crown Council held at Sinaia , arguing for Romania 's neutrality in World War I. Two years later , he took part in the Crown Council meeting at Cotroceni Palace , voting for Romania 's entry into the war on the side of the Allies of World War I .",
  'tags': 'o o o o o o o o o o o o o o b-organisation i-organisation o o b-location o o o b-country o o o b-event i-event i-event o o o o o o o o o b-organisation i-organisation o o b-location i-location o o o b-country o o o o o o o o o o b-organisation i-organisation i-organisation i-organisation i-organisation o',
  'gpt_response': "In the summer of 1914 , he took part in the meetings of the <organization>Crown Council</organization> held at <location>Sinaia</location> , arguing for <country>Romania</country>'s neutrality in <event>World War I</event>. Two years later , he took part in the <organization>Crown Council</organization> meeting at <location>Cotroceni Pa

## Few-Shot

In [ ]:
instuctions_examples = f"{instructions}\n{examples.strip()}"
print(instuctions_examples)

You are an information extraction assistant. Your task is to identify and label named entities in the input sentence according to the following types: politician, person, organization, politicalparty, event, election, country, location, and misc.Rewrite the entire sentence without changing or omitting any words. Surround each identified entity with a tag indicating its type, using the format <type>...</type>. For example, tag a politician as <politician>...</politician>.Only apply tags to complete spans of meaning, not to partial words or fragments. Do not tag anything that does not clearly belong to one of the listed types. Output only the fully tagged sentence, do not include any additional text.
Examples:
        1) Sentence: Brazil's major cities began to resemble 1932-33 Berlin with its street battles between the Communist Party of Germany and the Nazi Party.
        Output: <country>Brazil</country>'s major cities began to resemble 1932-33 <location>Berlin</location> with its str

In [ ]:
results = []
engine = "gpt-4.1-nano"
max_token = 512
temperature = 0.0  # Lower temp for consistency

for idx, (sentence, tag) in tqdm(dataset.iterrows(), total=len(dataset)):
  response = openai_client.chat.completions.create(
      model=engine,
      messages=[{"role": "system", "content": instuctions_examples},
                {"role": "user", "content": f"Sentence: {sentence}"}],
      max_tokens=max_token,
      temperature=temperature,
  ).choices[0].message.content.strip()
  results.append({
      'sentence': sentence,
      'tags': tag,
      'gpt_response': response
  })

pd.DataFrame(results).to_csv(f"{folder}/ner_few_shot_results.csv", index=False)

100%|██████████| 651/651 [12:15<00:00,  1.13s/it]


In [ ]:
results

[{'sentence': "In the summer of 1914 , he took part in the meetings of the Crown Council held at Sinaia , arguing for Romania 's neutrality in World War I. Two years later , he took part in the Crown Council meeting at Cotroceni Palace , voting for Romania 's entry into the war on the side of the Allies of World War I .",
  'tags': 'o o o o o o o o o o o o o o b-organisation i-organisation o o b-location o o o b-country o o o b-event i-event i-event o o o o o o o o o b-organisation i-organisation o o b-location i-location o o o b-country o o o o o o o o o o b-organisation i-organisation i-organisation i-organisation i-organisation o',
  'gpt_response': "In the summer of 1914 , he took part in the meetings of the <organization>Crown Council</organization> held at <location>Sinaia</location> , arguing for <country>Romania</country>'s neutrality in <event>World War I</event>. Two years later , he took part in the <organization>Crown Council</organization> meeting at <location>Cotroceni Pa

# SENTIMENT ANALYSIS

In [19]:
instructions = (
    "You are a sentiment classification assistant. Your task is to classify the sentiment of the sentence. The possibile sentiments are: positive or negative."
    "Output only one of the allowed relation types: positive or negative. Do not include punctuation, explanation, or formatting."
)
examples = ("""Examples:
1) Sentence: "Do not the Government’s proposals introduce some quite substantial financial risks for local authorities... councils in the very tough position of deciding who are the deserving poor and who are not?" Output: Negative
2) Sentence: "It is a pleasure to follow the right hon Member... the Minister's ID card proposals do not meet that threshold..." Output: Positive
3) Sentence: "Today’s debate is not just about reform of the health service; it is about democracy, accountability and transparency... That is a disgrace, and we should all support the motion today." Output: Negative
""")

In [20]:
print(instructions)

You are a sentiment classification assistant. Your task is to classify the sentiment of the sentence. The possibile sentiments are: positive or negative.Output only one of the allowed relation types: positive or negative. Do not include punctuation, explanation, or formatting.


In [21]:
dataset = pd.read_csv(f"{folder}/sentiment_analysis/test.csv")
len(dataset)

3346

In [22]:
dataset.head(3)

,text,label
0,"The important words are ""akin to"".We have had ...",0
1,"The Minister says that it is a minor matter, b...",0
2,May I reciprocate by thanking my hon Friend fo...,1


## Zero-Shot

In [12]:
results = []
engine = "gpt-4.1-nano"
max_token = 50
temperature = 0.0  # Lower temp for consistency

for idx, (sentence, label) in tqdm(dataset.iterrows(), total=len(dataset)):
  response = openai_client.chat.completions.create(
      model=engine,
      messages=[{"role": "system", "content": instructions},
                {"role": "user", "content": f"Sentence: {sentence}"}],
      max_tokens=max_token,
      temperature=temperature,
  ).choices[0].message.content.strip()
  results.append({
      'sentence': sentence,
      'label': label,
      'gpt_response': response
  })

pd.DataFrame(results).to_csv(f"{folder}/sentiment_analysis_zero_shot_results.csv", index=False)

100%|██████████| 3346/3346 [20:57<00:00,  2.66it/s]


In [13]:
results[0]

{'sentence': 'The important words are "akin to".We have had a good debate. My hon Friends have made excellent contributions but, as my hon Friend the Under-Secretary of State for Scotland has commented, it has been our pleasure to sit and watch a tableau of—That would be a rather cruel way to characterise it. Perhaps it would be fairer to say that a range of views was expressed.I have barely started, but if the hon Gentleman insists.I should not have bothered. I want to focus attention on the practical effects of the amendments, which would make changes that, as several hon Members pointed out, would alter the Bill\'s nature and character. They would introduce a series of discrepancies in the provisions, which would make it unworkable. The amendments that the hon Member for Gainsborough  tabled would allow siblings to form civil partnerships. The amendments that the hon Member for Christchurch  tabled would allow unmarried parents and children and unmarried siblings to form such partne

## Few-Shot

In [24]:
instuctions_examples = f"{instructions}\n{examples.strip()}"
print(instuctions_examples)

You are a sentiment classification assistant. Your task is to classify the sentiment of the sentence. The possibile sentiments are: positive or negative.Output only one of the allowed relation types: positive or negative. Do not include punctuation, explanation, or formatting.
Examples:
1) Sentence: "Do not the Government’s proposals introduce some quite substantial financial risks for local authorities... councils in the very tough position of deciding who are the deserving poor and who are not?" Output: Negative
2) Sentence: "It is a pleasure to follow the right hon Member... the Minister's ID card proposals do not meet that threshold..." Output: Positive
3) Sentence: "Today’s debate is not just about reform of the health service; it is about democracy, accountability and transparency... That is a disgrace, and we should all support the motion today." Output: Negative


In [25]:
results = []
engine = "gpt-4.1-nano"
max_token = 50
temperature = 0.0  # Lower temp for consistency

for idx, (sentence, label) in tqdm(dataset.iterrows(), total=len(dataset)):
  response = openai_client.chat.completions.create(
      model=engine,
      messages=[{"role": "system", "content": instuctions_examples},
                {"role": "user", "content": f"Sentence: {sentence}"}],
      max_tokens=max_token,
      temperature=temperature,
  ).choices[0].message.content.strip()
  results.append({
      'sentence': sentence,
      'label': label,
      'gpt_response': response
  })

pd.DataFrame(results).to_csv(f"{folder}/sentiment_analysis_few_shot_results.csv", index=False)

100%|██████████| 3346/3346 [20:41<00:00,  2.70it/s]


In [ ]:
results[0]

# RELATION CLASSIFICATION

In [27]:
instructions = (
        "You are a relation classification assistant. Your task is to determine the type of argumentative relation between two argument spans in the sentence below. The two arguments are separated by a period."
        "Output only one of the allowed relation types: support, attack, or neither. Do not include punctuation, explanation, or formatting."
    )
examples = (
        """Examples:
        1) Sentence: "We're not going to support the $300 billion tax cut that they have for corporate America and the very wealthy. It didn't meet my test."
        Output: neither

        2) Sentence: "They do have stakes in it. What we need now is a president who understands how to bring these other countries together to recognize their stakes in this."
        Output: support

        3) Sentence: "I respect the belief about life and when it begins. I can't take what is an article of faith for me and legislate it for someone who doesn't share that article of faith, whether they be agnostic, atheist, Jew, Protestant, whatever."
        Output: attack"""
    )

In [31]:
print(instructions)

You are a relation classification assistant. Your task is to determine the type of argumentative relation between two argument spans in the sentence below. The two arguments are separated by a period.Output only one of the allowed relation types: support, attack, or neither. Do not include punctuation, explanation, or formatting.


In [33]:
dataset = pd.read_csv(f"{folder}/relation_classification/test.csv")
len(dataset)

5102

In [34]:
dataset.head()

,link_type,linked,text
0,0,1,we want to do is commit more forces anywhere. ...
1,0,1,We need to call upon Syria to release the thre...
2,1,1,that's not the way this country ought to be ru...
3,1,1,if we remain dependent on a source of energy t...
4,2,0,This strategy requires additional troops. I do...


In [ ]:
for idx, row in tqdm(dataset.iterrows(), total=len(dataset)):
  print(row)

## Zero-Shot

In [37]:
results = []
engine = "gpt-4.1-nano"
max_token = 50
temperature = 0.0  # Lower temp for consistency

for idx, row in tqdm(dataset.iterrows(), total=len(dataset)):
  sentence = row.text
  link_type = row.link_type
  response = openai_client.chat.completions.create(
      model=engine,
      messages=[{"role": "system", "content": instructions},
                {"role": "user", "content": f"Sentence: {sentence}"}],
      max_tokens=max_token,
      temperature=temperature,
  ).choices[0].message.content.strip()
  results.append({
      'sentence': sentence,
      'link_type': link_type,
      'gpt_response': response
  })

pd.DataFrame(results).to_csv(f"{folder}/relation_classification_zero_shot_results.csv", index=False)

100%|██████████| 5102/5102 [28:54<00:00,  2.94it/s]


In [38]:
results[0]

{'sentence': 'we want to do is commit more forces anywhere. What I want the President to do, and I think he may have done it, his last statement, call for an unconditional end of the violence',
 'link_type': 0,
 'gpt_response': 'support'}

## Few-Shot

In [39]:
instuctions_examples = f"{instructions}\n{examples.strip()}"
print(instuctions_examples)

You are a relation classification assistant. Your task is to determine the type of argumentative relation between two argument spans in the sentence below. The two arguments are separated by a period.Output only one of the allowed relation types: support, attack, or neither. Do not include punctuation, explanation, or formatting.
Examples:
        1) Sentence: "We're not going to support the $300 billion tax cut that they have for corporate America and the very wealthy. It didn't meet my test."
        Output: neither

        2) Sentence: "They do have stakes in it. What we need now is a president who understands how to bring these other countries together to recognize their stakes in this."
        Output: support

        3) Sentence: "I respect the belief about life and when it begins. I can't take what is an article of faith for me and legislate it for someone who doesn't share that article of faith, whether they be agnostic, atheist, Jew, Protestant, whatever."
        Output: at

In [40]:
results = []
engine = "gpt-4.1-nano"
max_token = 50
temperature = 0.0  # Lower temp for consistency

for idx, row in tqdm(dataset.iterrows(), total=len(dataset)):
  sentence = row.text
  link_type = row.link_type
  response = openai_client.chat.completions.create(
      model=engine,
      messages=[{"role": "system", "content": instuctions_examples},
                {"role": "user", "content": f"Sentence: {sentence}"}],
      max_tokens=max_token,
      temperature=temperature,
  ).choices[0].message.content.strip()
  results.append({
      'sentence': sentence,
      'link_type': link_type,
      'gpt_response': response
  })

pd.DataFrame(results).to_csv(f"{folder}/relation_classification_few_shot_results.csv", index=False)

100%|██████████| 5102/5102 [26:35<00:00,  3.20it/s]
